# -Day 1- 搭建开发环境

 转链接https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432170937506ecfb2f6adf8e4757939732f3e32b781c000

# -Day 2- 编写Web App骨架

用aiohttp写一个基本的app.py

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import asyncio, os, json, time
from datetime import datetime

from aiohttp import web

In [3]:
def index(request):
    return web.Response(body=b'<h1>Awesome</h1>')

@asyncio.coroutine
def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('Get', '/', index)
    srv = yield from loop.create_server(app.make_handler(), '127.0.0.1', 9000)
    logging.info('server started at http://127.0.0.1:9000...')
    return srv

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

/home/dell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: loop argument is deprecated
  
/home/dell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Application.make_handler(...) is deprecated, use AppRunner API instead
  
INFO:root:server started at http://127.0.0.1:9000...
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:02:08 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:02:26 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:04:02 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"


# -Day 3-编写ORM

Object Relational Mapping 对象关系映射

## 创建连接池

In [4]:
import asyncio, os, json, time

@asyncio.coroutine
def create_pool(loop, **kw):
    logging.info('create database connection pool...')
    global __pool
    __pool = yield from aiomysql.create_pool(
        host=kw.get('host', 'localhost'),
        port=kw.get('port', 3306),
        user=kw['user'],
        password=kw['password'],
        db=kw['db'],
        charset=kw.get('charset','utf8'),
        autocommit=kw.get('autocommit', True),
        maxsize=kw.get('maxsize', 10),
        minsize=ke.get('minsize', 1),
        loop=loop
    )

## Select
要执行SELECT语句，我们用select函数执行，需要传入SQL语句和SQL参数

In [5]:
@asyncio.coroutine
def select(sql, args, size=None):
    log(sql, args)
    global __pool
    with (yield from __pool) as conn:
        cur = yield from conn.cursor(aiomysql.DictCursor)
        #将SQL语句的占位符'?'替换为MySql的占位符'%s'
        yield from cur.execute(sql.replace('?', '%s'), args or ())
        if size:
            #通过fetchmany()获取最多指定数量的记录
            rs = yield from cur.fetchmany(size)
        else:
            #否则，通过fetall()获取所有记录
            rs = yield from cur.fetchall()
        yield from cur.close()
        logging.info('rows returned: %s' % len(rs))
        return rs

## Insert, Update, Delete
要执行INSERT、UPDATE、DELETE语句，可以定义一个通用的execute()函数，因为这3种SQL的执行都需要相同的参数，以及返回一个整数表示影响的行数

In [6]:
@asyncio.coroutine
def execute(sql, args):
    log(sql)
    with (yield from __pool) as conn:
        try:
            cur = yield from conn.cursor()
            yield from cur.execute(sql.replace('?', '%s'), args)
            affected = cur.rowcount
            yield from cur.close()
        except BaseException as e:
            raise
        return affected

## 定义Model
首先定义所有ORM映射的基类Model

In [11]:
def create_args_string(num):
    L = []
    for n in range(num):
        L.append('?')
    return ', '.join(L)

# Field 和各种Field子类
class Field(object):

    def __init__(self, name, column_type, primary_key, default):
        self.name = name
        self.column_type = column_type
        self.primary_key = primary_key
        self.default = default

    def __str__(self):
        return '<%s, %s:%s>' % (self.__class__.__name__, self.column_type, self.name)

class StringField(Field):

    def __init__(self, name=None, primary_key=False, default=None, ddl='varchar(100)'):
        super().__init__(name, ddl, primary_key, default)

class BooleanField(Field):

    def __init__(self, name=None, default=False):
        super().__init__(name, 'boolean', False, default)

class IntegerField(Field):

    def __init__(self, name=None, primary_key=False, default=0):
        super().__init__(name, 'bigint', primary_key, default)

class FloatField(Field):

    def __init__(self, name=None, primary_key=False, default=0.0):
        super().__init__(name, 'real', primary_key, default)

class TextField(Field):

    def __init__(self, name=None, default=None):
        super().__init__(name, 'text', False, default)

  


In [14]:
class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        #排除Model类本身
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        #获取table名称
        tableName = attrs.get('__table__', None) or name
        logging.info('found model: %s (table: %s)' % (name, tableName))
        #获取所有的Field和主键名
        mappings = dict()
        fields = []
        primaryKey = None
        for k, v in attrs.items():
            if isinstance(v, Field):
                logging.info(' found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
                if v.primary_key:
                    #找到主键:
                    if primaryKey:
                        raise RuntimeError('Duplicate primary key for field: %s' % k)
                    primaryKey = k
                else:
                    fields.append(k)
        if not primaryKey:
            raise RuntimeError('Primary key not found.')
        for k in mappings.keys():
            attrs.pop(k)
        escaped_fields = list(map(lambda f: "'%s'" % f, fields))
        attrs['__mappings__'] = mappings                  #保存属性和列的映射关系
        attrs['__table__'] = tableName
        attrs['__primary_key__'] = primaryKey             #主键属性名
        attrs['__fields__'] = fields                      #除主键外的属性名
        # 构造默认的SELECT, INSERT, UPDATE 和DELETE语句：
        attrs['__select__'] = "select '%s', %s from '%s'" % (primaryKey, ', '.join(escaped_fields), tableName)
        attrs['__insert__'] = "insert into '%s' (%s, '%s') values (%s)" \
                               % (tableName, ', '.join(escaped_fields), primaryKey, create_args_string(len(escaped_fields) + 1))
        attrs['__update__'] = "update '%s' set %s where '%s' = ?" \
                               % (tableName, ', '.join(map(lambda f: "'%s'=?" % (mappings.get(f).name or f), fields)), primaryKey)
        attrs['__delete__'] = "delete from '%s' where '%s'=?" % (tableName, primaryKey)
        return type.__new__(cls, name, bases, attrs)
            

class Model(dict, metaclass=ModelMetaclass):
    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def getValue(self, key):
        return getattr(self, key, None)

    def getValueOrDefault(self, key):
        value = getattr(self, key, None)
        if value is None:
            field = self.__mappings__[key]
            if field.default is not None:
                value = field.default() if callable(field.default) else field.default
                logging.debug('using default value for %s: %s' % (key, str(value)))
                setattr(self, key, value)
        return value

    @classmethod
    async def findAll(cls, where=None, args=None, **kw):
        ' find objects by where clause. '
        sql = [cls.__select__]
        if where:
            sql.append('where')
            sql.append(where)
        if args is None:
            args = []
        orderBy = kw.get('orderBy', None)
        if orderBy:
            sql.append('order by')
            sql.append(orderBy)
        limit = kw.get('limit', None)
        if limit is not None:
            sql.append('limit')
            if isinstance(limit, int):
                sql.append('?')
                args.append(limit)
            elif isinstance(limit, tuple) and len(limit) == 2:
                sql.append('?, ?')
                args.extend(limit)
            else:
                raise ValueError('Invalid limit value: %s' % str(limit))
        rs = await select(' '.join(sql), args)
        return [cls(**r) for r in rs]

    @classmethod
    async def findNumber(cls, selectField, where=None, args=None):
        ' find number by select and where. '
        sql = ['select %s _num_ from `%s`' % (selectField, cls.__table__)]
        if where:
            sql.append('where')
            sql.append(where)
        rs = await select(' '.join(sql), args, 1)
        if len(rs) == 0:
            return None
        return rs[0]['_num_']

    @classmethod
    async def find(cls, pk):
        ' find object by primary key. '
        rs = await select('%s where `%s`=?' % (cls.__select__, cls.__primary_key__), [pk], 1)
        if len(rs) == 0:
            return None
        return cls(**rs[0])

    async def save(self):
        args = list(map(self.getValueOrDefault, self.__fields__))
        args.append(self.getValueOrDefault(self.__primary_key__))
        rows = await execute(self.__insert__, args)
        if rows != 1:
            logging.warn('failed to insert record: affected rows: %s' % rows)

    async def update(self):
        args = list(map(self.getValue, self.__fields__))
        args.append(self.getValue(self.__primary_key__))
        rows = await execute(self.__update__, args)
        if rows != 1:
            logging.warn('failed to update by primary key: affected rows: %s' % rows)

    async def remove(self):
        args = [self.getValue(self.__primary_key__)]
        rows = await execute(self.__delete__, args)
        if rows != 1:
            logging.warn('failed to remove by primary key: affected rows: %s' % rows)
            
    async def insert(self):
        args = list(map(self.getValue, self.__fields__))
        args.append(self.getValue(self.__primary_key__))
        rows = await execute(self.__insert__, args)
        

## ORM

In [15]:
#from www.orm import Model, StringField, IntegerField

class User(Model):
    __table__ = 'users'
    
    id = IntegerField(primary_key=True)
    name = StringField()

INFO:root:found model: User (table: users)
INFO:root: found mapping: id ==> <IntegerField, bigint:None>
INFO:root: found mapping: name ==> <StringField, varchar(100):None>


In [16]:
# 创建实例:
user = User(id=123, name='Yuan')
# 存入数据库
user.save
#查询所有User对象:\
users = User.findAll()

# -Day 4-编写Model


## 用Model表示三个需要的表 
有了ORM，我们就可以把Web App需要的3个表用Model表示出来

In [ ]:
import time, uuid

from www.orm import Model, StringField, BooleanField, FloatField, TextField

def next_id():
    return '%015d%s000' % (int(time.time() * 1000), uuid.uuid4().hex)

class User(Model):
    __table__ = 'users'
    
    id = StringField(primary_key=True, default=next_id, ddl='varchar(50)')
    email = StringField(ddl='varchar(50)')
    passwd = StringField(ddl='varchar(50)')
    admin = BooleanField()
    name = StringField(ddl='varchar(50)')
    image = StringField(ddl='varchar(500)')
    created_at = FloatField(default=time.time)
    
class Blog(Model):
    __table__ = 'blogs'
    
    id = StringField(primary_key=True, default=next_id, ddl='varchar(50)')
    user_id = StringField(ddl='varchar(50)')
    user_name = StringField(ddl='varchar(50)')
    user_image = StringField(ddl='varchar(500)')
    name = StringField(ddl='varchar(50)')
    summary = StringField(ddl='varchar(200)')
    content = TextField()
    created_at = FloatField(default=time.time)
    
class Comment(Model):
    __table__ = 'comments'
    
    id = StringField(primary_key=True, default=next_id, ddl='varchar(50)')
    blog_id = StringField(ddl='varchar(50)')
    user_id = StringField(ddl='varchar(50)')
    user_name = StringField(ddl='varchar(50)')
    user_image = StringField(ddl='varchar(500)')
    content = TextField()
    created_at = FloatField(default=time.time)
    


## 初始化数据库表
如果表的数量很少，可以手写创建表的SQL脚本：

In [ ]:
#-- schema.sql

drop database if exists awesome;

create database awesome;

use awesome;

grant select, insert, update, delete on awesome.* to 'www-data'@'localhost' identified by 'www-data';

create table users(
    'id' varchar(50) not null,
    'email' varchar(50) not null,
    'passwd' varchar(50) not null,
    'admin' bool not null,
    'name' varchar(50) not null,
    'image' varchar(500) not null,
    'created_at' real not null,
    unique key 'idx_email' ('email'),
    key 'idx_created_at' ('created_at'),
    primary key ('id')
)engine=innodb default charset=utf8;

create table blogs(
    'id' varchar(50) not null,
    'user_id' varchar(50) not null,
    'user_name' varchar(50) not null,
    'user_image' varchar(500) not null,
    'name' varchar(50) not null,
    'summary' varchar(200) not null,
    'content' mediumtext not null,
    'created_at' real not null,
)engine=innodb default charset=utf8;

## 编写数据访问代码
接下来，就可以真正开始编写代码操作对象了。比如，对于User对象，我们就可以做如下操作：

In [1]:
import asyncio
import www.orm as orm
from www.models import User, Blog, Comment

async def test(loop):
    await orm.create_pool(loop=loop, user='www-data', password='www-data', db='awesome')
    
    u = User(name='Test', email='test@example.com', passwd='1234567890', image='about:blank')
    
    await u.save()

loop = asyncio.get_event_loop()
loop.run_until_complete(test(loop))
print('Test finished.')
loop.close()
    
    

Test finished.


# -Day 5-编写Web框架